# Project Phase 2

### Yolo Object detection for person, mask, and no-mask

In [1]:
import cv2
import numpy as np
import glob
import random
import time

In [2]:
load_dir = "C:/Users/Dell/DataScience/Lums/CS5310/MiniProject/Task2"

In [3]:
# Load Yolo
yolo_net = cv2.dnn.readNet( load_dir + "/weights3hrs/mask-yolov3_10000.weights", load_dir + "/weights3hrs/mask-yolov3.cfg")

layer_names = yolo_net.getLayerNames()

output_layers = [layer_names[i[0] - 1] for i in yolo_net.getUnconnectedOutLayers()]


In [4]:
classes = ["Person", "mask", "noMask"]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

colors = [0, 122, 255] # color of boxes
# print(colors)
# print(len(colors))

In [5]:
cameras = {
    "Camera-1": load_dir + "/Videos/Stream1Recording.mp4",
    "Camera-2": load_dir + "/Videos/campus4-c1.mp4",
#     "Camera-3": load_dir + "/Videos/campus7-c1.mp4"    
}

In [6]:
def Get_Available_Cams(cams_detail,starting = 390):
    available_cams = {}

    for key, video_address in cams_detail.items():
        cam = cv2.VideoCapture(video_address)
        if cam.isOpened() == True:
            cam.set(1,starting)
            available_cams[key] = cam
            
    return available_cams

In [7]:
def Object_Detection(current_frame):
    
    for key in list(current_frame.keys()):
        
        # Image
        img = current_frame[key]
        height, width, channels = img.shape

        # Detecting objects
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

        yolo_net.setInput(blob)
        outs = yolo_net.forward(output_layers)

        # Showing informations on the screen
        class_ids = []
        confidences = []
        boxes = [] 
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.7:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        font = cv2.ACCESS_MASK
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                color = colors[class_ids[i]]
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
                cv2.putText(img, label + str(np.around(confidences[i],2)*100) + "%", (x, y-5), font, 0.4, color, 1)

        cv2.imshow(key, img)
        cv2.waitKey(1)        

In [8]:
def Read_Image_Yolo(available_cams):
    
    cam_keys = list(available_cams.keys())

    while len(cam_keys) == len(available_cams):
        
        current_frame = {}        
        for i , key in enumerate(cam_keys):
            is_open , frame = available_cams[key].read()
            if is_open == True:
                current_frame[key] = frame
            else:
                cam_keys.pop(i)
                
        if len(cam_keys) == len(available_cams):
            Object_Detection(current_frame)
            
    for key in available_cams.keys():
        available_cams[key].release()
 

In [9]:
cams_avail = Get_Available_Cams(cameras)
Read_Image_Yolo(cams_avail)
cv2.destroyAllWindows()

In [10]:
cv2.destroyAllWindows()